### GET DATA

In [1]:
import pandas as pd
import os
os.getcwd()

'/Users/deepakm/mywork/mynlp/txnstmt'

In [2]:
dtxn=pd.read_csv('flat_transactions.csv')
dtxn.head()

,kde_id,id,country,description,transactionDate,transactionAmount,transactionAmountCurrency,bankTransactionFlowType,currentBalanceAmount,currentBalanceCurrency,accountIdentifier,bankIdentifier,amountInFunctionalCurrencyAmount,amountInFunctionalCurrencyCurrency,currentBalanceInFunctionalCurrencyAmount,currentBalanceInFunctionalCurrencyCurrency,bankDataSource
0,10048,1,NL,AUTOMAT S6EE6080 0063 Karte3 27.06.2015 UM 18:...,2015-06-29,-400.00,EUR,OUTFLOW,-6006.540039,EUR,170331,Bawag,-400.00,EUR,-6006.540039,EUR,titanium-db
1,10048,2,NL,---- 0103619245 / 200003507629 ----Rechnungsnr...,2015-06-29,-38.08,EUR,OUTFLOW,-6406.540039,EUR,170331,Bawag,-38.08,EUR,-6406.540039,EUR,titanium-db
2,10048,257,NL,150351,2015-06-29,228.64,EUR,INFLOW,-48527.500000,EUR,170511,Bawag,228.64,EUR,-48527.500000,EUR,titanium-db
3,10048,258,NL,"Kundennummer 000000086189, Hutchison Drei Aust...",2015-06-29,-30.22,EUR,OUTFLOW,-48298.859380,EUR,170511,Bawag,-30.22,EUR,-48298.859380,EUR,titanium-db
4,10048,259,NL,---- 0103619245 / 200003768285 ----Rechnungsnr...,2015-06-29,-34.98,EUR,OUTFLOW,-48329.078130,EUR,170511,Bawag,-34.98,EUR,-48329.078130,EUR,titanium-db


In [3]:
data=dtxn['description'].tolist()
print(len(data))
data[:20]

419446


['AUTOMAT S6EE6080 0063 Karte3 27.06.2015 UM 18:22  ',
 '---- 0103619245 / 200003507629 ----Rechnungsnr. 295114742 120 ',
 '150351  ',
 'Kundennummer 000000086189, Hutchison Drei Austria  ',
 '---- 0103619245 / 200003768285 ----Rechnungsnr. 295114744 732 ',
 'Rg.Nr. 150390  ',
 'R 150352  ',
 'Renr: 150402      /KR-Nr: 002910-00000-00/RE-Nr:   201500 0583/Rein. Ba                                   lkon Wies enhofer Ultra',
 'Renr: 150403      /KR-Nr: 002980-00000-00/RE-Nr:   201500 0499/Mont. Ba                                   nk, Ultra s',
 'Renr: 150406      /KR-Nr: 007870-00448-65/RE-Nr:   201500 1533/Sanierun                                   g Zufahrt sstra\x9de',
 'Renr: 150400      /KR-Nr: 006860-00000-00/RE-Nr:   201500 1508/Ultras Z                                   ylinder v ermessenWhg11',
 'Kd.Nr:234545 Re.Nr:151119  ',
 '  ',
 '  ',
 '  ',
 '  ',
 'Entgelt f\x9dr Kontof\x9dhrung  ',
 'Entgelt Kontoauszug                                1 Stk.  ',
 'Entgelt f\x9dr automati

In [4]:
data=data[:10000]

### GENERATE EMBEDDINGS

#### Embeddings from Universal Sentence Encoder
https://tfhub.dev/google/universal-sentence-encoder-large/5

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
# embed=hub.load("/pretrained_models/universal-sentence-encoder_4")
embeddings=embed(data)

#### Embeddings from BERT models

#### Models tuned for cosine-similarity will prefer the retrieval of short documents, 
#### while models tuned for dot-product will prefer the retrieval of longer documents. 
#### Depending on your task, the models of the one or the other type are preferable.

In [9]:
# !pip install sentence_transformers
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("sentence-transformers/bert-base-nli-mean-tokens")

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('msmarco-distilbert-base-v3')

query_embedding = model.encode('How big is London')
passage_embedding = model.encode('London has 9,787,426 inhabitants at the 2011 census')

print("Similarity:", util.pytorch_cos_sim(query_embedding, passage_embedding))

In [11]:
# embeddings = model.encode(data, show_progress_bar=True)

#### Look at Embeddings

In [ ]:
embeddings

### REDUCE DIMENSIONS OF EMBEDDINGS FOR BETTER CLUSTERING RESULTS

In [ ]:
from umap import UMAP
umap_embeddings = UMAP(n_neighbors=10,n_components=20, metric='cosine').fit_transform(embeddings)

In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(copy=False)
umap_embeddings=normalizer.fit_transform(umap_embeddings)

### CLUSTER THE RESULTS
https://nbviewer.jupyter.org/github/scikit-learn-contrib/hdbscan/blob/master/notebooks/How%20HDBSCAN%20Works.ipynb

#### Clustering by HDBSCAN

In [ ]:
from hdbscan import HDBSCAN,RobustSingleLinkage
clusterer = HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)
print("Number of Clusters found: ",clusterer.labels_.max())
print("Cluster Probabilities: ",clusterer.probabilities_)
# print("Cluster Validity: ",clusterer.relative_validity_)
# clusterer = RobustSingleLinkage(cut=0.125, k=7)
# cluster_labels = clusterer.fit_predict(data)

# hierarchy = clusterer.cluster_hierarchy_
# alt_labels = hierarchy.get_clusters(0.100, 5)
# hierarchy.plot()



In [ ]:
# PLOT THE RESULTS BY REDUCING DIM TO 2
import matplotlib.pyplot as plt

# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = clusterer.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()

### Aggregate Text by Clusters

In [ ]:
docs_df = pd.DataFrame(data, columns=["Doc"])
docs_df['Clust'] = clusterer.labels_
docs_df['Doc_ID'] = range(len(docs_df))
print(docs_df.head())
docs_per_cluster = docs_df.groupby(['Clust'], as_index = False).agg({'Doc': ' '.join})
docs_per_cluster.head()

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))

### TOP N WORDS IN CLUSTER

In [ ]:
def extract_top_n_words_per_cluster(tf_idf, count, docs_per_cluster, n=10):
    words = count.get_feature_names()
    labels = list(docs_per_cluster.Clust)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_cluster_sizes(df):
    cluster_sizes = (df.groupby(['Clust'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Clust": "Clust", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return cluster_sizes

top_n_words = extract_top_n_words_per_cluster(tf_idf, count, docs_per_cluster, n=20)
cluster_sizes = extract_cluster_sizes(docs_df)
print("Number of Clusters Found: ",cluster_sizes.shape[0])
cluster_sizes.head(10)

In [ ]:
top_n_words[1][:10]
top_n_words

#### Reductions of Cluster by Combining Similar Clusters

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(20):
    # Calculate cosine similarity
    similarities = cosine_similarity(tf_idf.T)
    np.fill_diagonal(similarities, 0)

    # Extract label to merge into and from where
    topic_sizes = docs_df.groupby(['Clust']).count().sort_values("Doc", ascending=False).reset_index()
    topic_to_merge = topic_sizes.iloc[-1].Clust
    topic_to_merge_into = np.argmax(similarities[topic_to_merge + 1]) - 1

    # Adjust topics
    docs_df.loc[docs_df.Clust == topic_to_merge, "Clust"] = topic_to_merge_into
    old_topics = docs_df.sort_values("Clust").Clust.unique()
    map_topics = {old_topic: index - 1 for index, old_topic in enumerate(old_topics)}
    docs_df.Topic = docs_df.Clust.map(map_topics)
    docs_per_topic = docs_df.groupby(['Clust'], as_index = False).agg({'Doc': ' '.join})

    # Calculate new topic words
    m = len(data)
    tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m)
    top_n_words = extract_top_n_words_per_cluster(tf_idf, count, docs_per_topic, n=10)

topic_sizes = extract_cluster_sizes(docs_df)
print("Number of Topics after Reduction: ",topic_sizes.shape[0])
topic_sizes.head(10)

In [ ]:
top_n_words[-1][:5]

In [ ]:
topw={}
for k,v in top_n_words.items():
    topw[k]=[i[0] for i in v]

topicdt=pd.DataFrame.from_dict(topw).T.reset_index()
topicdt.head()

In [ ]:
topicdt.to_csv("topicdt_transf.csv")

#### Clustering using K-Means

In [ ]:
X=umap_embeddings

In [ ]:
#WE WILL USE K MEANS CLUSTERING ON DESCRIPTIONS
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
#FIND OUT OPTIMAL NUMBER OF CLUSTERS
#METHOD 1: Elbow curve based on SSE 
sse = {}
slh={}
for k in range(2, 50):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X)
    sse[k] = kmeans.inertia_ 
#    this is to find silhouette score
    label = kmeans.labels_
    slh[k] = silhouette_score(X, label, metric='euclidean')
#PLOT THE GRAPHS
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
# plt.show()
#METHOD 2: Using  silhouette score: higher the better
plt.figure(figsize=(15, 5))
plt.plot(list(slh.keys()), list(slh.values()))
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Score")
# plt.show()

num_clus=max(slh,key=slh.get)

In [ ]:
#####################################################
#GET num_clus FROM ABOVE 2 METHODS AND FIT THE FINAL CLSUTER
num_clus = 20
model = KMeans(n_clusters=num_clus, init='k-means++', max_iter=100, n_init=2)
model.fit(X)
cluster_assignment = model.labels_
clustered_sentences = [[] for i in range(num_clus)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(ques[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [ ]:
clustered_data={}
for i, cluster in enumerate(clustered_sentences):
    clustered_data[i+1]=cluster


# clustered_data
clustered_data=pd.DataFrame.from_dict(clustered_data,orient = 'index').T
clustered_data.columns=["Clus_"+str(col) for col in clustered_data.columns]
clustered_data.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster
import time
%matplotlib inline
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}
def plot_clusters(data, algorithm, args, kwds):
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=24)
    plt.text(-0.5, 0.7, 'Clustering took {:.2f} s'.format(end_time - start_time), fontsize=14)

plot_clusters(umap_embeddings, cluster.KMeans, (), {'n_clusters':20})

# plot_clusters(umap_embeddings, cluster.AffinityPropagation, (), {'preference':-5.0, 'damping':0.95})
# plot_clusters(umap_embeddings, cluster.MeanShift, (0.175,), {'cluster_all':False})

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
 
# data = fetch_20newsgroups(subset='all')['data']

model = BERTopic("distilbert-base-nli-stsb-mean-tokens", verbose=True)
topics, probabilities = model.fit_transform(data)

In [ ]:
print(len(topics))

In [ ]:
# model.get_topic(1)
pd.DataFrame.from_dict(model.get_topics()).T

In [ ]:
# Make sure to input the probabilities of a single document!
model.visualize_distribution(probabilities[15])

In [ ]:
from transformers import DistilBertTokenizer, RobertaTokenizer
distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model
# roberta = 'roberta-base-uncase'

# Defining DistilBERT tokonizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,max_length=128, pad_to_max_length=True)
# Defining RoBERTa tokinizer
# tokenizer = RobertaTokenizer.from_pretrained(roberta, do_lower_case=True, add_special_tokens=True,max_length=128, pad_to_max_length=True)


def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=128, pad_to_max_length=True, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=6)
config.output_hidden_states = False
transformer_model = TFDistilBertForSequenceClassification.from_pretrained(distil_bert, config = config)[0]

input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
X = transformer_model(input_ids, input_masks_ids)
model = tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)



distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(192, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(6, activation='softmax')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
    layer.trainable = False

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(50, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(6, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
    layer.trainable = False